In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
img = cv2.imread('./test_image/test8.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

h, w, c = img.shape

if w > 1000:
    new_w = 1000
    ar = w/h
    new_h = int(new_w/ar)
    img = cv2.resize(img, (new_w, new_h), interpolation = cv2.INTER_AREA)
plt.imshow(img)

In [ ]:
def thresholding(image):
    img_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(img_gray,80,255,cv2.THRESH_BINARY_INV)
    plt.imshow(thresh, cmap='gray')
    return thresh

thresh_img = thresholding(img)

In [ ]:
#dilation
kernel = np.ones((3,85), np.uint8)
dilated = cv2.dilate(thresh_img, kernel, iterations = 1)
plt.imshow(dilated, cmap='gray');

In [ ]:
(contours, heirarchy) = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
sorted_contours_lines = sorted(contours, key = lambda ctr : cv2.boundingRect(ctr)[1]) # (x, y, w, h)

In [ ]:
img2 = img.copy()
cropped_images = []

for ctr in sorted_contours_lines:
    x, y, w, h = cv2.boundingRect(ctr)
    cv2.rectangle(img2, (x, y), (x+w, y+h), (40, 100, 250), 2)
    cropped_region = img2[y:y+h, x:x+w] 
    cropped_images.append(cropped_region)

plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY), cmap='gray') 
plt.show()
cv2.imshow(f'Cropped Image {1}', cv2.cvtColor(cropped_images[13], cv2.COLOR_BGR2GRAY))

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:

gray_img = cv2.cvtColor(cropped_images[13], cv2.COLOR_BGR2GRAY)
ones_array = np.where(gray_img == 255, 1, 0)

ones_img = np.uint8(ones_array * 255)


cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
from scipy.signal import find_peaks

vertical_hist = np.sum(ones_img, axis=0, keepdims=True)
peaks, _ = find_peaks(vertical_hist[0], height=3500)


# แสดงกราฟ
plt.plot(vertical_hist[0])
plt.plot(peaks, vertical_hist[0][peaks], 'x', color='red')
plt.title('Vertical Histogram Projection with Peaks')
plt.xlabel('Column')
plt.ylabel('Number of White Pixels')
plt.show()



In [ ]:
print(peaks)

In [ ]:
def character_segmentation(word_segment, character_threshold=3500, character_width=10, character_gap=5):
    vertical_projection = np.sum(word_segment, axis=0)
    character_peaks, _ = find_peaks(vertical_projection, height=character_threshold)

    segmented_characters = []

    for peak_column in character_peaks:
        middle_column = peak_column
        start_column = max(0, middle_column - character_width // 2 - character_gap)
        end_column = min(word_segment.shape[1], middle_column + character_width // 2)
        character_segment = word_segment[:, start_column:end_column]
        segmented_characters.append(character_segment)

    return segmented_characters




In [ ]:
segmented_characters = character_segmentation(ones_img, character_threshold=50)

characters_list = []

for i, char_segment in enumerate(segmented_characters):
    characters_list.append(char_segment)

for i, char_segment in enumerate(characters_list):
    print(f'Character {i + 1}:')
    print(char_segment)
    print('\n')

cv2.imshow('Character 1', characters_list[3])

cv2.waitKey(0)

cv2.destroyAllWindows()


In [ ]:

def vowel_segmentation(word_segment, character_threshold=3500, character_height=10, character_gap=5):
    horizontal_projection = np.sum(word_segment, axis=1)
    character_peaks, _ = find_peaks(horizontal_projection, height=character_threshold)

    segmented_characters = []

    for peak_row in character_peaks:
        middle_row = peak_row
        start_row = max(0, middle_row - character_height // 2 - character_gap)
        end_row = min(word_segment.shape[0], middle_row + character_height // 2)
        character_segment = word_segment[start_row:end_row, :]
        segmented_characters.append(character_segment)

    return segmented_characters


def char_segmentation(image, character_threshold=3500, character_height=10, character_gap=5):
    horizontal_projection = np.sum(image, axis=1)
    character_peaks, _ = find_peaks(horizontal_projection, height=character_threshold)

    segmented_characters = []

    for i in range(len(character_peaks) - 1):
        start_row = character_peaks[i]
        end_row = character_peaks[i + 1]
        character_segment = image[start_row:end_row, :]
        segmented_characters.append(character_segment)
    segmented_characters.append(image[character_peaks[-1]:, :])

    return segmented_characters

In [ ]:
vo = characters_list[3]
segmented_characters = char_segmentation(characters_list[3])

for i, char_segment in enumerate(segmented_characters):
    cv2.imshow(f'Character {i + 1}', char_segment)


cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
segmented_vowel = vowel_segmentation(vo)

for i, char_segment in enumerate(segmented_vowel):
    cv2.imshow(f'Character {i + 1}', char_segment)


cv2.waitKey(0)
cv2.destroyAllWindows()